In [1]:
import numpy as np
from sympy import *
xi=Symbol ('xi',real=True) # we can use this variable to compute functions and derivatives
#                          # e.g., diff(sqrt(abs(xi)), xi)

import holoviews as hv; hv.extension('bokeh', 'plotly', logo=False)
import panel as pn;     pn.extension()
from   panel.interact import interact

<div style="float:center;width:100%;text-align: center;"><strong style="height:60px;color:darkred;font-size:40px;">Gradient Descent</strong></div>

# 1. Find a Minimum

## 1.1 The Idea of the Gradient Descent Algorithm

Consider the following 3 functions

In [2]:
x=np.arange(-5,5,step=0.1)

h1 = hv.Curve((x, x**2+10)).opts(title="Global Minimum")
h2 = hv.Curve((x,(x-4)*(x-1)*(x-1)*(x+5)/200)).opts(title="Local Minima")
h3 = hv.Curve((x,np.sqrt(np.abs(x)))).opts(title="Non-differentiable")

h  = h1*hv.Scatter(([0],[10]))+\
     h2*hv.Scatter(([-3.33,3.08],[-1.15,-0.16]))+\
     h3*hv.Scatter(([0],[0]))
pn.Row(h.opts(shared_axes=False)\
        .opts("Curve", width=200, height=200, tools=["hover"])\
        .opts("Scatter", size=8, color="red"))

Row
    [0] HoloViews(Layout)

We would like to find a minimum of a given function $y = f(x)$.

**Idea:** start at some point on the curve and **move a short distance downward,** then repeat.<br>
$\qquad$ The tangent line is well defined and approximates the function locally provided<br>
$\qquad$ **the function is differentiable.** This immediately excludes the third function<br>
$\qquad$ from consideration for this algorithm.

For a differentiable function $y = f(x), f : \mathbb{R}^n \rightarrow \mathbb{R}$, the steepest descent direction is $\nabla f(x)$.<br>
$\qquad$ Start with some value $x$ and move downhill by some distance determined by a **parameter** $\mathbf{\mu}$,<br>
$\qquad$ i.e., **move to $\mathbf{x - \mu \nabla f(x)}$.**

$\qquad$ **as we approach a minimum $\mathbf{\nabla f(x) \rightarrow 0}$, so the motion should cease!**

____
Let's try to implement this idea.

In [3]:
def gradient_descent( f, df, x, mu, n_max, tol = 1.e-6):
    # f:      the function y=f(x)
    # df:     the derivative function y = f'(x)
    # x:      some starting value for the iteration
    # mu:     the distance to move along the slope of the line
    # n_max:  the number of times to try, bail if the function does not converge
    # tol:    smallest the change in height before declaring victory

    y      = f(x)               # starting height
    vals_x = [x]
    vals_y = [y]

    for i in range(n_max):      # don't try more than n times at most
        delta_x = -mu*df(x)     # move downhill (hence the minus sign)
        x_new   = x + delta_x
        y_new   = f(x_new)
        vals_x.append(x_new); vals_y.append(y_new)

        if np.linalg.norm(delta_x) < tol: return (x_new,y_new), (vals_x, vals_y)   # converged
        x,y = x_new,y_new

    return None, (vals_x,vals_y)  # failed to converge

In [4]:
def show_gradient_descent(hf, f, df, x, mu, n_max, ylim, tol=1e-4):
    sol, vals = gradient_descent(f,df,x,mu,n_max,tol)

    h = hv.Curve(vals).opts(xlim=(-5,5), ylim=ylim)*\
        hv.Scatter(vals).opts(size=4, color="red", tools=['hover'])
    if sol is not None:
        h = h*hv.Scatter((sol)).opts(size=5, color="black")
    hy = hv.Curve(vals[1], "iteration index", "y")\
           .opts(ylim=ylim,height=350, title="y versus Iteration Index")

    return (hf*h).opts(width=450,height=350)+hy

## 1.2 Two Scalar Function Examples

In [5]:
def f1(x):  return x*x+10
def df1(x): return 2*x

interact( lambda x,mu: show_gradient_descent(h1,f1,df1,x,mu,50,(9,36)),
          x  = pn.widgets.FloatSlider(name = "initial x", start=-5,  end=5,  step=.1,   value=4.5),
          mu = pn.widgets.FloatSlider(name = "mu",        start=0.01,end=1.03,step=0.005,value=0.2) )

Column
    [0] Column
        [0] FloatSlider(end=5, name='initial x', start=-5, value=4.5)
        [1] FloatSlider(end=1.03, name='mu', start=0.01, step=0.005, value=0.2)
    [1] Row
        [0] HoloViews(Layout, name='interactive01649')

This problem is well behaved: **Since the parameter $\mu$ multiplies the slope**<br>
$\qquad$ **the steps get smaller as the slope approaches 0.**

Let's try the second example.

In [6]:
def f2(x):  return (x**4 - x**3 - 21*x*x + 41*x - 20)/200
def df2(x): return (4*x**3-3*x**2-42*x+41)/200
interact( lambda x,mu: show_gradient_descent(h2,f2,df2,x,mu,50, (-1.5,0.8)),
          x  = pn.widgets.FloatSlider(name="initial x", start=-5,  end=5,step=.1,  value=4.5),
          mu = pn.widgets.FloatSlider(name="mu",        start=0.01,end=5,step=0.01,value=0.2) )

Column
    [0] Column
        [0] FloatSlider(end=5, name='initial x', start=-5, value=4.5)
        [1] FloatSlider(end=5, name='mu', start=0.01, step=0.01, value=0.2)
    [1] Row
        [0] HoloViews(Layout, name='interactive01959')

**Remark:**
* In machine learning application, the parameter $\mu$ is called the **learning rate**
* if $\mu$ is too small, the scheme may fail to converge
* if $\mu$ is too large, we see the system repeatedly overshoot the location of the minimum
* Depending on the initial guess $x$ and the value of $\mu$, the scheme will move toward either minumum.

## 1.3 What about the Cusp Function?

In [7]:
def f3(x): return np.sqrt(np.abs(x))
def df3(x): # clamp the derivative away from ±∞
    if x < 0.:
        return -0.5/f3( np.min([x,-1e-6]))
    else:
        return 0.5/f3(np.max([x,1e-6]))

interact( lambda x,mu: show_gradient_descent(h3,f3,df3,x,mu,50, (-0.5,2.5)),#.opts(xlim=(-5,5), ylim=(-1.5,1)),
          x  = pn.widgets.FloatSlider(name="initial x", start=-5,end=5,step=.1,value=4.5),
          mu = pn.widgets.FloatSlider(name="mu",        start=0.01,end=5,step=0.01,value=0.2) )

Column
    [0] Column
        [0] FloatSlider(end=5, name='initial x', start=-5, value=4.5)
        [1] FloatSlider(end=5, name='mu', start=0.01, step=0.01, value=0.2)
    [1] Row
        [0] HoloViews(Layout, name='interactive02214')

## 1.4 Convex Functions

**Definition:** A function $f(x)$ is **convex** on an interval $[a,b]$<br>
$\qquad$ if for any two points $x_1$ and $x_2$ in $[a,b]$ and any $\lambda$ where $0<\lambda<1$,

$\qquad f(\lambda x_1 + (1-\lambda) x_2) \le \lambda f(x_1)+(1-\lambda)f(x_2)$. 


**Remark:** this definition easily generalize to function $f: \mathbb{R}^n \rightarrow \mathbb{R}$

In essence, this definition says that all points on any line segment connecting two points on a curve<br>
$\qquad$ must lie on or above the curve:

In [8]:
def show_line(x,f,slope):
    def line(some_x):
        return f(x) + slope*(some_x-x)
    return hv.Curve( ([-5,5], [line(-5),line(5)])).opts(line_width=0.8,color="red")*\
           hv.Scatter((x,f(x))).opts(size=4,color="red")

interact( lambda x,theta: (h1.opts(ylim=(   9, 36),width=350,height=250, title="A Convex Function")*\
                           show_line(x,f1,np.tan(np.pi*theta/180))+\
                           h2.opts(ylim=(-1.5,0.8),width=350,height=250, title="Not a Convex Function")*\
                           show_line(x,f2,np.tan(np.pi*theta/180))).opts(shared_axes=False),
          x     = pn.widgets.FloatSlider(name="initial x", start=-5,end=5,step=.1,value=-4.5),
          theta = pn.widgets.FloatSlider(name="theta",     start=-88,end=88,step=1,value=0)
)

Column
    [0] Column
        [0] FloatSlider(end=5, name='initial x', start=-5, value=-4.5)
        [1] FloatSlider(end=88, name='theta', start=-88, step=1)
    [1] Row
        [0] HoloViews(Layout, name='interactive02469')

**Theorem:** For differentiable convex functions, the following three properties are equivalent:
* $x$ is a local minimum of $f(x)$
* $x$ is a global minimum of $f(x)$
* $\nabla f(x)=0$

In other words, **any critical point** of the complex function $f(x)$ in $[a,b]$ **is a global minimum.**

**Advantages and disadvantages of gradient descent**
* Simple algorithm that is easy to implement
* Can be very fast for smooth functions
* However, it’s often slow
* Cannot handle non-differentiable functions (biggest downside)

# 2. Least Mean Squares Example

When we studied Least Mean Squares, we saw that the [Normal Equation](20_LengthOrthogonality.ipynb) $A^t A x = A^t b$ problem<br>
$\qquad$ is equivalent to the minimization problem

$\qquad
x^* = \arg\min_x { \lVert A x - b\rVert }\;\;\Leftrightarrow \;\; \arg\min_x { \lVert A x - b\rVert^2 }
$

We are looking for a minimum of the function $f(x) = \min_x \Vert A x - b\Vert^2$

Using the chain rule, the gradient is found to be $\nabla f(x) = A^t ( A x - b )$

In [9]:
def show_objective_surface(f,lims=(-3,3), step=0.25):
    x_vals = np.arange(*lims,step)
    xx,yy  = np.meshgrid(x_vals,x_vals)
    z      = list(map( f, np.array([xx.ravel(),yy.ravel()]).T ))
    return hv.TriSurface((xx.ravel(),yy.ravel(), z ))

def show_objective_surface_contours(f,lims=(-3,3),n=101,l=[5,10,20,50]):
    from holoviews.operation import contours as hv_contours
    v   = np.linspace(*lims,n)
    x,y = np.meshgrid(v, v)
    z   = np.array(list(map( f, np.array([x.ravel(),y.ravel()]).T ))).reshape(x.shape)

    #h.opts( hv.opts.Polygons(cmap='fire'))
    return hv.Image((v,v,z)), hv_contours(hv.Image((v,v,z)), levels=l, filled=True, overlaid=False)

In [10]:
%output backend="plotly" size=120
A    = np.array( [[1,0.1],[0.4,-1]])
b    = np.array( [[-0.2],[0.1]])
x_0  = np.array( [[-3],[-3]])

def fa(x):  return np.linalg.norm( A @ x + b)**2
def dfa(x): return 2*A.T @ ( A @ x - b)
# Remark: using summation convention and a_k the k'th col of A, these are
#F    =   ((a_k ⋅ a_i) x_i x_k  -  2 (a_k ⋅ b) x_k  # remove unneeded constant term
#dF_k = 2 ((a_k ⋅ a_i) x_i      -    (a_k ⋅ b)
# this yields
#def fa(x):
#    Ax = A @ x
#    return Ax.dot( Ax - 2*b )

ha = show_objective_surface( fa )
pn.Row(ha)

Row
    [0] HoloViews(TriSurface)

In [11]:
%output backend="bokeh" size=100

hmap, h_contours = show_objective_surface_contours(fa,lims=(-3,3), l=[.3,1,5,10,20,40])

def show_gradient_descent_on_contour(hf, f, df, x, mu, n_max, ylim, tol=1e-4):
    sol, steps = gradient_descent(f,df,x,mu,n_max,tol)
    x_i = [p[0,0] for p in steps[0]]
    y_i = [p[1,0] for p in steps[0]]

    if sol is not None: sol = np.round(sol[0].T[0], 3)

    return pn.Row(
        hf.opts(width=350,height=350,xlim=(-3,3),ylim=(-3,3),title=f"Contour Lines {sol}" )\
        *hv.Curve(  (x_i, y_i)).opts(color="red")\
        *hv.Scatter((x_i, y_i)).opts(tools=['hover'],color="red", size=4),
        hv.Curve( steps[1], "index", "z" ).opts(height=350, title="y versus Iteration Index")
    )
interact( lambda mu: show_gradient_descent_on_contour(h_contours,fa,dfa,x_0,mu,100,(0,50), tol=1e-10),
          mu = pn.widgets.FloatSlider(name="mu",        start=0.01,end=1,step=0.01,value=0.2) )

Column
    [0] Column
        [0] FloatSlider(name='mu', start=0.01, step=0.01, value=0.2)
    [1] Row
        [0] Row
            [0] HoloViews(Overlay, height=350, sizing_mode='fixed', width=350)
            [1] HoloViews(Curve, height=350, sizing_mode='fixed', width=300)